In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from itertools import combinations, groupby
from collections import Counter
from config import DATASET

In [2]:
from config import *

# Data read in.
orders = pd.read_csv(DATASET)

# check status
print(f'total sales: {orders.shape[0]}')
print(f'unique orders count: {len(orders.order_id.unique())}')
print(f'unique products count: {len(orders.product_id.unique())}')

# Dropout 'add_to_cart_order' and 'reordered' columns.
orders = orders.drop(columns=['add_to_cart_order', 'reordered'], errors='ignore')

# Only select 1000 orders
# orders = orders.head(1000)

ValueError: Multiple files found in ZIP file. Only one file per ZIP: ['order_products__prior.csv', '__MACOSX/', '__MACOSX/._order_products__prior.csv']

In [94]:
orders

,order_id,product_id
0,2,33120
1,2,28985
2,2,9327
3,2,45918
4,2,30035
...,...,...
995,109,33651
996,109,36066
997,109,40310
998,110,5876


In [97]:
from typing import List, FrozenSet, Set, Dict
from itertools import combinations

itemset: List[frozenset] = [frozenset({item}) for item in orders.product_id.unique()]

transactions: List[list] = [
    orders
        .groupby('order_id')
        .get_group(oid)
        .product_id
        .to_list()
            for oid in orders.order_id.unique()
    ]

In [ ]:
#根据候选k项集和最小支持度，生成频繁k项集

def get_support(transactions: List[list], itemsets: List[frozenset], min_sup: float):
    r"""Get support values for itemsets.

    transactions: List[list]
    """
    support = dict()#定义项集-频数字典，用来存储每个项集key对应的频数value
    for trans in transactions:
        for itemset in itemsets:
            #判断项集是否在记录中出现
            if itemset.issubset(trans):
                support[itemset] = support.get(itemset, 0) + 1
    total = len(transactions)
    return {k: v/total for k, v in support.items() if v/total >= min_sup}


L1 = ck_to_lk(transactions, itemset, 0.005)
L1

In [106]:
#频繁K项集组合生成候选K+1项集
def get_combined_itemsets(ori_itemsets: List[frozenset], item_size: int):
    r"""Get combined itemsets.
    """
    #保存所有组合之后的候选k+1项集
    result_itemsets = set()
    ori_size = len(ori_itemsets)
    if ori_size > 1:
        k = len(ori_itemsets[0])
        for i, j in combinations(range(ori_size), item_size):
            t = ori_itemsets[i] | ori_itemsets[j]
            if len(t) == k+1: result_itemsets.add(t)
    return result_itemsets
c2 = get_combined_itemsets(list(L1.keys()), 2)
c3 = get_combined_itemsets(list(L1.keys()), 3)
c3

ValueError: too many values to unpack (expected 2)